In [31]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
import tensorflow as tf

def load_and_preprocess_data(data_dir, img_size=(224, 224), batch_size=32):
    # Define class names based on your dataset structure
    class_names = ['healthy', 'red_spider_mite', 'Rust']

    # Load and preprocess images
    train_images, train_labels = [], []
    test_images, test_labels = [], []

    for split in ['Train', 'test']:
        for class_idx, class_name in enumerate(class_names):
            if split == 'Train':
                class_dir = os.path.join(data_dir, split, class_name.replace('_', ' '))
            else:
                class_dir = os.path.join(data_dir, split, class_name)
            
            if not os.path.exists(class_dir):
                print(f"Warning: Directory not found: {class_dir}")
                continue
            
            for img_name in os.listdir(class_dir):
                img_path = os.path.join(class_dir, img_name)
                img = Image.open(img_path).convert('RGB')
                img = img.resize(img_size)
                img_array = np.array(img) / 255.0  # Normalize pixel values
                if split == 'Train':
                    train_images.append(img_array)
                    train_labels.append(class_idx)
                else:
                    test_images.append(img_array)
                    test_labels.append(class_idx)

    train_images, train_labels = np.array(train_images), np.array(train_labels)
    test_images, test_labels = np.array(test_images), np.array(test_labels)

    # Split training data into train and validation sets
    X_train, X_val, y_train, y_val = train_test_split(train_images, train_labels, test_size=0.2, stratify=train_labels, random_state=42)

    # Create TensorFlow datasets
    train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(1000).batch(batch_size)
    val_ds = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(batch_size)
    test_ds = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).batch(batch_size)

    return train_ds, val_ds, test_ds, class_names

# Usage example
if __name__ == "__main__":
    data_dir = r"C:/Users/91911/OneDrive/Desktop/IEEE/COFFEE/dataset"
    train_ds, val_ds, test_ds, class_names = load_and_preprocess_data(data_dir)
    
    print(f"Number of classes: {len(class_names)}")
    print(f"Class names: {class_names}")
    
    # Print information about the datasets
    for dataset, name in zip([train_ds, val_ds, test_ds], ["Training", "Validation", "Test"]):
        images, labels = next(iter(dataset))
        print(f"{name} dataset:")
        print(f"  Image shape: {images.shape}")
        print(f"  Label shape: {labels.shape}")
        print(f"  Types: {images.dtype}, {labels.dtype}")
        print(f"  Min-Max pixel values: {tf.reduce_min(images).numpy():.2f}, {tf.reduce_max(images).numpy():.2f}")

Number of classes: 3
Class names: ['healthy', 'red_spider_mite', 'Rust']
Training dataset:
  Image shape: (32, 224, 224, 3)
  Label shape: (32,)
  Types: <dtype: 'float64'>, <dtype: 'int32'>
  Min-Max pixel values: 0.00, 1.00
Validation dataset:
  Image shape: (32, 224, 224, 3)
  Label shape: (32,)
  Types: <dtype: 'float64'>, <dtype: 'int32'>
  Min-Max pixel values: 0.00, 1.00
Test dataset:
  Image shape: (32, 224, 224, 3)
  Label shape: (32,)
  Types: <dtype: 'float64'>, <dtype: 'int32'>
  Min-Max pixel values: 0.00, 1.00


In [16]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.applications.efficientnet import EfficientNetB0

def create_cnn_model(input_shape=(224, 224, 3), num_classes=3):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

def apply_traditional_augmentations(ds):
    data_augmentation = tf.keras.Sequential([
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.2),
        layers.RandomZoom(0.2),
    ])
    return ds.map(lambda x, y: (data_augmentation(x, training=True), y),
                  num_parallel_calls=tf.data.AUTOTUNE)

def train_and_evaluate_model(model, train_ds, val_ds, test_ds, epochs=10):
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    history = model.fit(train_ds, validation_data=val_ds, epochs=epochs)
    
    test_loss, test_acc = model.evaluate(test_ds)
    print(f"Test accuracy: {test_acc:.4f}")
    
    return history, test_acc

# Load the data (assuming you've already run the data preparation script)
data_dir = r"C:/Users/91911/OneDrive/Desktop/IEEE/COFFEE/dataset"
train_ds, val_ds, test_ds, class_names = load_and_preprocess_data(data_dir)

# Apply traditional augmentations to the training data
augmented_train_ds = apply_traditional_augmentations(train_ds)

# Create and train the CNN model
cnn_model = create_cnn_model()
cnn_history, cnn_test_acc = train_and_evaluate_model(cnn_model, augmented_train_ds, val_ds, test_ds)

print(f"CNN Model Test Accuracy: {cnn_test_acc:.4f}")


Epoch 1/10


c:\Users\91911\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


18/18 ━━━━━━━━━━━━━━━━━━━━ 13s 594ms/step - accuracy: 0.3933 - loss: 3.2412 - val_accuracy: 0.4071 - val_loss: 1.0496
Epoch 2/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 25s 1s/step - accuracy: 0.4263 - loss: 1.0683 - val_accuracy: 0.4071 - val_loss: 1.0399
Epoch 3/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 30s 2s/step - accuracy: 0.3881 - loss: 1.0607 - val_accuracy: 0.3643 - val_loss: 1.0335
Epoch 4/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 35s 2s/step - accuracy: 0.4152 - loss: 1.0802 - val_accuracy: 0.4000 - val_loss: 1.0150
Epoch 5/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 44s 2s/step - accuracy: 0.3901 - loss: 1.0344 - val_accuracy: 0.4286 - val_loss: 1.0433
Epoch 6/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 10s 530ms/step - accuracy: 0.3897 - loss: 1.0479 - val_accuracy: 0.4000 - val_loss: 1.0306
Epoch 7/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 31s 2s/step - accuracy: 0.4301 - loss: 1.0323 - val_accuracy: 0.3857 - val_loss: 1.0296
Epoch 8/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 26s 1s/step - accuracy: 0.4543 - loss: 1.0319 - val_accuracy: 0.3714 - val_loss: 1.01

In [18]:
# Save the trained CNN model
cnn_model.save('cnn_coffee_leaf_model.h5')
print("CNN model saved as 'cnn_coffee_leaf_model.h5'")

CNN model saved as 'cnn_coffee_leaf_model.h5'


In [17]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.applications.efficientnet import EfficientNetB0
import numpy as np

def create_transfer_learning_model(base_model, num_classes=3):
    base_model.trainable = False
    inputs = layers.Input(shape=(224, 224, 3))
    x = base_model(inputs, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(128, activation='relu')(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    model = tf.keras.Model(inputs, outputs)
    return model

def mixup_data(x, y, alpha=0.2):
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = tf.shape(x)[0]
    index = tf.random.shuffle(tf.range(batch_size))

    mixed_x = lam * x + (1 - lam) * tf.gather(x, index)
    mixed_y = lam * y + (1 - lam) * tf.gather(y, index)
    return mixed_x, mixed_y

def train_and_evaluate_model(model, train_ds, val_ds, test_ds, epochs=20, use_mixup=False):
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    callbacks = [
        tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True),
        tf.keras.callbacks.ReduceLROnPlateau(factor=0.2, patience=3)
    ]
    
    if use_mixup:
        train_ds = train_ds.map(lambda x, y: mixup_data(x, tf.one_hot(y, depth=3)))
    else:
        train_ds = train_ds.map(lambda x, y: (x, tf.one_hot(y, depth=3)))
    
    val_ds = val_ds.map(lambda x, y: (x, tf.one_hot(y, depth=3)))
    test_ds = test_ds.map(lambda x, y: (x, tf.one_hot(y, depth=3)))
    
    history = model.fit(train_ds, validation_data=val_ds, epochs=epochs, callbacks=callbacks)
    
    test_loss, test_acc = model.evaluate(test_ds)
    print(f"Test accuracy: {test_acc:.4f}")
    
    return history, test_acc

# Load the data (assuming you've already run the data preparation script)
data_dir = r"C:/Users/91911/OneDrive/Desktop/IEEE/COFFEE/dataset"
train_ds, val_ds, test_ds, class_names = load_and_preprocess_data(data_dir)

# Apply traditional augmentations to the training data
augmented_train_ds = apply_traditional_augmentations(train_ds)

# Create and train the advanced models
models = {
    "ResNet50": create_transfer_learning_model(ResNet50(weights='imagenet', include_top=False)),
    "MobileNetV2": create_transfer_learning_model(MobileNetV2(weights='imagenet', include_top=False)),
    "EfficientNetB0": create_transfer_learning_model(EfficientNetB0(weights='imagenet', include_top=False))
}

results = {}

for model_name, model in models.items():
    print(f"\nTraining {model_name} with traditional augmentation:")
    _, acc = train_and_evaluate_model(model, augmented_train_ds, val_ds, test_ds)
    results[f"{model_name}_traditional"] = acc
    
    print(f"\nTraining {model_name} with MixUp augmentation:")
    _, acc = train_and_evaluate_model(model, augmented_train_ds, val_ds, test_ds, use_mixup=True)
    results[f"{model_name}_mixup"] = acc

# Print final results
print("\nFinal Results:")
for model_name, acc in results.items():
    print(f"{model_name}: {acc:.4f}")


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 13s 0us/step


C:\Users\91911\AppData\Local\Temp\ipykernel_7292\844133934.py:66: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  "MobileNetV2": create_transfer_learning_model(MobileNetV2(weights='imagenet', include_top=False)),


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step

Training ResNet50 with traditional augmentation:
Epoch 1/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 238s 11s/step - accuracy: 0.3331 - loss: 1.1561 - val_accuracy: 0.4000 - val_loss: 1.0945 - learning_rate: 0.0010
Epoch 2/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 225s 13s/step - accuracy: 0.3703 - loss: 1.1617 - val_accuracy: 0.4071 - val_loss: 1.0232 - learning_rate: 0.0010
Epoch 3/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 256s 14s/step - accuracy: 0.4155 - loss: 1.0643 - val_accuracy: 0.4000 - val_loss: 1.0051 - learning_rate: 0.0010
Epoch 4/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 337s 18s/step - accuracy: 0.3793 - loss: 1.0691 - val_accuracy: 0.4071 - val_loss: 1.0062 - learning_rate: 0.0010
Epoch 5/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 290s 15s/step - accuracy: 0.3952 - loss: 1.0724 - val_accuracy: 0.4071 - val_loss: 0.9844 - learning_rate: 0.0010
Epoch 6/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 33s 2s/step - accuracy: 0.4264 - loss: 1.0601 - val_

In [19]:
# Find the best model
best_model_key = max(results, key=results.get)
best_model_name = best_model_key.split('_')[0]
best_model = models[best_model_name]

In [23]:

print(f"The best model '{best_model_name}' ")

The best model 'MobileNetV2' 


In [24]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
import numpy as np

def create_fine_tuned_model(num_classes=3):
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    
    # Unfreeze the last few layers
    for layer in base_model.layers[-20:]:
        layer.trainable = True
    
    inputs = layers.Input(shape=(224, 224, 3))
    x = base_model(inputs)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    model = tf.keras.Model(inputs, outputs)
    
    return model

def train_and_evaluate_model(model, train_ds, val_ds, test_ds, epochs=30, use_mixup=True):
    model.compile(optimizer=tf.keras.optimizers.Adam(1e-4),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    callbacks = [
        tf.keras.callbacks.EarlyStopping(patience=7, restore_best_weights=True),
        tf.keras.callbacks.ReduceLROnPlateau(factor=0.2, patience=5)
    ]
    
    if use_mixup:
        train_ds = train_ds.map(lambda x, y: mixup_data(x, tf.one_hot(y, depth=3)))
    else:
        train_ds = train_ds.map(lambda x, y: (x, tf.one_hot(y, depth=3)))
    
    val_ds = val_ds.map(lambda x, y: (x, tf.one_hot(y, depth=3)))
    test_ds = test_ds.map(lambda x, y: (x, tf.one_hot(y, depth=3)))
    
    history = model.fit(train_ds, validation_data=val_ds, epochs=epochs, callbacks=callbacks)
    
    test_loss, test_acc = model.evaluate(test_ds)
    print(f"Test accuracy: {test_acc:.4f}")
    
    return history, test_acc

def create_ensemble(models):
    inputs = layers.Input(shape=(224, 224, 3))
    outputs = [model(inputs) for model in models]
    ensemble_output = layers.Average()(outputs)
    ensemble_model = tf.keras.Model(inputs=inputs, outputs=ensemble_output)
    return ensemble_model

# Load the data and apply augmentations (assuming you've already defined these functions)
data_dir = r"C:/Users/91911/OneDrive/Desktop/IEEE/COFFEE/dataset"
train_ds, val_ds, test_ds, class_names = load_and_preprocess_data(data_dir)
augmented_train_ds = apply_traditional_augmentations(train_ds)

# Fine-tune MobileNetV2
fine_tuned_model = create_fine_tuned_model()
print("\nFine-tuning MobileNetV2:")
_, fine_tuned_acc = train_and_evaluate_model(fine_tuned_model, augmented_train_ds, val_ds, test_ds)

# Create and train multiple models for ensemble
ensemble_models = [create_fine_tuned_model() for _ in range(3)]
for i, model in enumerate(ensemble_models):
    print(f"\nTraining ensemble model {i+1}:")
    train_and_evaluate_model(model, augmented_train_ds, val_ds, test_ds)

# Create and evaluate ensemble
ensemble_model = create_ensemble(ensemble_models)
print("\nEvaluating ensemble model:")
ensemble_model.compile(loss='categorical_crossentropy', metrics=['accuracy'])
_, ensemble_acc = ensemble_model.evaluate(test_ds.map(lambda x, y: (x, tf.one_hot(y, depth=3))))

print("\nFinal Results:")
print(f"Fine-tuned MobileNetV2: {fine_tuned_acc:.4f}")
print(f"Ensemble Model: {ensemble_acc:.4f}")

# Save the best model
best_model = fine_tuned_model if fine_tuned_acc > ensemble_acc else ensemble_model
best_model.save('best_coffee_leaf_modelww.h5')
print(f"\nBest model saved as 'best_coffee_leaf_modelww.h5'")


Fine-tuning MobileNetV2:
Epoch 1/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 114s 3s/step - accuracy: 0.4153 - loss: 1.2498 - val_accuracy: 0.4429 - val_loss: 1.1738 - learning_rate: 1.0000e-04
Epoch 2/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 47s 3s/step - accuracy: 0.5731 - loss: 1.0628 - val_accuracy: 0.4786 - val_loss: 1.2149 - learning_rate: 1.0000e-04
Epoch 3/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 48s 3s/step - accuracy: 0.6241 - loss: 0.9020 - val_accuracy: 0.4571 - val_loss: 1.4645 - learning_rate: 1.0000e-04
Epoch 4/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 45s 2s/step - accuracy: 0.6812 - loss: 0.8750 - val_accuracy: 0.4571 - val_loss: 1.4652 - learning_rate: 1.0000e-04
Epoch 5/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 45s 2s/step - accuracy: 0.7171 - loss: 0.8350 - val_accuracy: 0.4286 - val_loss: 1.6974 - learning_rate: 1.0000e-04
Epoch 6/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 45s 2s/step - accuracy: 0.7397 - loss: 0.8176 - val_accuracy: 0.4214 - val_loss: 2.0845 - learning_rate: 1.0000e-04
Epoch 7/30
18/18 ━━━━━━━━━━━━━━━━━━━━ 79s 2s/step


Final Results:
Fine-tuned MobileNetV2: 0.5367
Ensemble Model: 0.5300


ValueError: Unable to synchronously create dataset (name already exists)

In [36]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2, ResNet50, EfficientNetB0
import numpy as np
from sklearn.utils import class_weight
import matplotlib.pyplot as plt

def load_and_preprocess_data(data_dir):
    img_size = (224, 224)
    batch_size = 32

    train_ds = tf.keras.utils.image_dataset_from_directory(
        data_dir,
        validation_split=0.2,
        subset="training",
        seed=123,
        image_size=img_size,
        batch_size=batch_size
    )
    val_ds = tf.keras.utils.image_dataset_from_directory(
        data_dir,
        validation_split=0.2,
        subset="validation",
        seed=123,
        image_size=img_size,
        batch_size=batch_size
    )

    test_ds = val_ds.take(5)
    val_ds = val_ds.skip(5)

    normalization_layer = layers.Rescaling(1./255)
    train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
    val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))
    test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y))

    return train_ds, val_ds, test_ds

def create_advanced_model(num_classes=3):
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    
    for layer in base_model.layers[-30:]:
        layer.trainable = True
    
    inputs = layers.Input(shape=(224, 224, 3))
    x = base_model(inputs)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    model = tf.keras.Model(inputs, outputs)
    
    return model

def advanced_augmentation(image, label):
    image = tf.cast(image, tf.float32)  # Ensure image is float32
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_brightness(image, max_delta=0.2)
    image = tf.image.random_contrast(image, lower=0.8, upper=1.2)
    image = tf.image.random_saturation(image, lower=0.8, upper=1.2)
    image = tf.image.random_hue(image, max_delta=0.2)
    return image, label

def mixup(image, label, alpha=0.2):
    batch_size = tf.shape(image)[0]
    lam = tf.random.uniform(shape=(), minval=0, maxval=alpha)
    index = tf.random.shuffle(tf.range(batch_size))
    mixed_image = lam * image + (1 - lam) * tf.gather(image, index)
    mixed_label = lam * label + (1 - lam) * tf.gather(label, index)
    return mixed_image, mixed_label

def custom_schedule(epoch, lr):
    if epoch < 10:
        return float(lr)
    else:
        return float(lr * tf.math.exp(-0.1))

def train_and_evaluate_model(model, train_ds, val_ds, test_ds, class_weights, epochs=50):
    lr_schedule = tf.keras.callbacks.LearningRateScheduler(custom_schedule)
    
    model.compile(optimizer=tf.keras.optimizers.Adam(1e-4),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    callbacks = [
        tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True),
        tf.keras.callbacks.ReduceLROnPlateau(factor=0.2, patience=5),
        lr_schedule
    ]
    
    # Apply advanced augmentation and mixup
    train_ds = train_ds.map(advanced_augmentation)
    train_ds = train_ds.map(lambda x, y: mixup(x, tf.one_hot(tf.cast(y, tf.int32), depth=3)))
    
    val_ds = val_ds.map(lambda x, y: (x, tf.one_hot(tf.cast(y, tf.int32), depth=3)))
    test_ds = test_ds.map(lambda x, y: (x, tf.one_hot(tf.cast(y, tf.int32), depth=3)))
    
    history = model.fit(
        train_ds, 
        validation_data=val_ds, 
        epochs=epochs, 
        callbacks=callbacks,
        class_weight=class_weights
    )
    
    test_loss, test_acc = model.evaluate(test_ds)
    print(f"Test accuracy: {test_acc:.4f}")
    
    return history, test_acc

# Main execution
data_dir = r"C:/Users/91911/OneDrive/Desktop/IEEE/COFFEE/dataset"
train_ds, val_ds, test_ds = load_and_preprocess_data(data_dir)

# Calculate class weights
y_train = np.concatenate([y for x, y in train_ds], axis=0)
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights = dict(enumerate(class_weights))

# Create and train the advanced model
advanced_model = create_advanced_model()
print("\nTraining advanced model:")
history, advanced_acc = train_and_evaluate_model(advanced_model, train_ds, val_ds, test_ds, class_weights)

# Plot training history
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.savefig('training_history.png')
plt.close()

print(f"\nAdvanced Model Accuracy: {advanced_acc:.4f}")

# Save the best model
advanced_model.save('best_coffee_leaf_model.h5')
print(f"\nBest model saved as 'best_coffee_leaf_model.h5'")
print(f"Training history plot saved as 'training_history.png'")

# Analyze misclassifications
test_images = np.concatenate([x for x, _ in test_ds], axis=0)
test_labels = np.concatenate([y for _, y in test_ds], axis=0)
predictions = advanced_model.predict(test_images)
misclassified = np.where(np.argmax(predictions, axis=1) != np.argmax(test_labels, axis=1))[0]

print(f"\nNumber of misclassified images: {len(misclassified)}")
print("Class distribution of misclassified images:")
class_names = ['healthy', 'red_spider_mite', 'Rust']
for class_idx, class_name in enumerate(class_names):
    count = np.sum(np.argmax(test_labels[misclassified], axis=1) == class_idx)
    print(f"{class_name}: {count}")

# Save some misclassified images for further analysis
num_images = min(5, len(misclassified))
for i in range(num_images):
    idx = misclassified[i]
    plt.imshow(test_images[idx])
    plt.title(f"True: {class_names[np.argmax(test_labels[idx])]}, Predicted: {class_names[np.argmax(predictions[idx])]}")
    plt.axis('off')
    plt.savefig(f'misclassified_{i}.png')
    plt.close()

print(f"\nSaved {num_images} misclassified images for analysis.")

Found 1000 files belonging to 2 classes.
Using 800 files for training.
Found 1000 files belonging to 2 classes.
Using 200 files for validation.

Training advanced model:
Epoch 1/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 75s 2s/step - accuracy: 0.4223 - loss: 1.2336 - val_accuracy: 0.6250 - val_loss: 0.8037 - learning_rate: 1.0000e-04
Epoch 2/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 62s 2s/step - accuracy: 0.5885 - loss: 0.8564 - val_accuracy: 0.5750 - val_loss: 0.7314 - learning_rate: 1.0000e-04
Epoch 3/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 71s 3s/step - accuracy: 0.6124 - loss: 0.7779 - val_accuracy: 0.7000 - val_loss: 0.5401 - learning_rate: 1.0000e-04
Epoch 4/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 73s 3s/step - accuracy: 0.7074 - loss: 0.6524 - val_accuracy: 0.8000 - val_loss: 0.4722 - learning_rate: 1.0000e-04
Epoch 5/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 55s 2s/step - accuracy: 0.7413 - loss: 0.5842 - val_accuracy: 0.7750 - val_loss: 0.4189 - learning_rate: 1.0000e-04
Epoch 6/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 56s 2s/step - accuracy


Advanced Model Accuracy: 0.8375


ValueError: Unable to synchronously create dataset (name already exists)

In [39]:
# Save using the new Keras format
advanced_model.save('ragha_coffee_leaf_model.keras')
print("\nBest model saved as 'best_coffee_leaf_model.keras'")


Best model saved as 'best_coffee_leaf_model.keras'


Found 1000 files belonging to 2 classes.
Using 800 files for training.
Found 1000 files belonging to 2 classes.
Using 200 files for validation.


TypeError: Singleton array array(<_MapDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>,
      dtype=object) cannot be considered a valid collection.

In [53]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetV2B0
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from sklearn.utils import class_weight

def create_advanced_model_v2(num_classes=3):
    base_model = EfficientNetV2B0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    
    for layer in base_model.layers[-30:]:
        layer.trainable = True
    
    inputs = layers.Input(shape=(224, 224, 3))
    x = base_model(inputs)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
    x = layers.Dropout(0.6)(x)
    x = layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
    x = layers.Dropout(0.6)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    model = tf.keras.Model(inputs, outputs)
    
    return model

def tf_random_beta(alpha, beta, shape=[]):
    x = tf.random.gamma(shape, alpha, 1.0)
    y = tf.random.gamma(shape, beta, 1.0)
    return x / (x + y)

def cutmix(image, label, alpha=1.0):
    batch_size = tf.shape(image)[0]
    image_size = tf.shape(image)[1]
    
    # Generate random bounding box
    lambda_ = tf_random_beta(alpha, alpha, shape=[])
    r_x = tf.random.uniform(shape=[], minval=0, maxval=image_size, dtype=tf.int32)
    r_y = tf.random.uniform(shape=[], minval=0, maxval=image_size, dtype=tf.int32)
    r_w = tf.cast(tf.cast(image_size, tf.float32) * tf.sqrt(1 - lambda_), tf.int32)
    r_h = tf.cast(tf.cast(image_size, tf.float32) * tf.sqrt(1 - lambda_), tf.int32)
    
    # Adjust bounding box to be within image
    x1 = tf.clip_by_value(r_x - r_w // 2, 0, image_size)
    y1 = tf.clip_by_value(r_y - r_h // 2, 0, image_size)
    x2 = tf.clip_by_value(r_x + r_w // 2, 0, image_size)
    y2 = tf.clip_by_value(r_y + r_h // 2, 0, image_size)
    
    # Create mask
    mask = tf.pad(tf.ones((y2-y1, x2-x1)), [[y1, image_size-y2], [x1, image_size-x2]])
    mask = tf.expand_dims(mask, axis=-1)
    
    # Apply CutMix
    image2 = tf.random.shuffle(image)
    label2 = tf.random.shuffle(label)
    mixed_image = image * (1 - mask) + image2 * mask
    mixed_label = label * (1 - lambda_) + label2 * lambda_
    
    return mixed_image, mixed_label

# def cosine_annealing_warmup(epoch, lr, total_epochs, warmup_epochs=5):
#     if epoch < warmup_epochs:
#         return lr * (epoch + 1) / warmup_epochs
#     else:
#         return lr * 0.5 * (1 + tf.math.cos(tf.constant(np.pi) * (epoch - warmup_epochs) / (total_epochs - warmup_epochs)))
    
def cosine_annealing_warmup(epoch, lr, total_epochs, warmup_epochs=5):
    if epoch < warmup_epochs:
        return lr * (epoch + 1) / warmup_epochs
    else:
        return lr * 0.5 * (1 + np.cos(np.pi * (epoch - warmup_epochs) / (total_epochs - warmup_epochs)))

# Modify the LearningRateScheduler to use this updated function
lr_schedule = tf.keras.callbacks.LearningRateScheduler(lambda epoch: float(cosine_annealing_warmup(epoch, 1e-3, epochs)))


def train_and_evaluate_model_v2(model, train_ds, val_ds, test_ds, class_weights, epochs=100):
    lr_schedule = tf.keras.callbacks.LearningRateScheduler(lambda epoch: cosine_annealing_warmup(epoch, 1e-3, epochs))
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    callbacks = [
        tf.keras.callbacks.EarlyStopping(patience=15, restore_best_weights=True),
        lr_schedule
    ]
    
    # Apply CutMix
    train_ds = train_ds.map(lambda x, y: cutmix(x, tf.one_hot(tf.cast(y, tf.int32), depth=3)))
    
    val_ds = val_ds.map(lambda x, y: (x, tf.one_hot(tf.cast(y, tf.int32), depth=3)))
    test_ds = test_ds.map(lambda x, y: (x, tf.one_hot(tf.cast(y, tf.int32), depth=3)))
    
    history = model.fit(
        train_ds, 
        validation_data=val_ds, 
        epochs=epochs, 
        callbacks=callbacks,
        class_weight=class_weights
    )
    
    test_loss, test_acc = model.evaluate(test_ds)
    print(f"Test accuracy: {test_acc:.4f}")
    
    return history, test_acc

def load_and_preprocess_data(data_dir, image_size=(224, 224), batch_size=32, val_split=0.2, test_split=0.1):
    dataset = tf.keras.preprocessing.image_dataset_from_directory(
        data_dir,
        image_size=image_size,
        batch_size=batch_size,
        label_mode='int',  # For integer labels
        validation_split=val_split + test_split,  # Total validation + test split
        subset='training',
        seed=123
    )

    val_test_ds = tf.keras.preprocessing.image_dataset_from_directory(
        data_dir,
        image_size=image_size,
        batch_size=batch_size,
        label_mode='int',
        validation_split=val_split + test_split,
        subset='validation',
        seed=123
    )

    # Split val_test_ds into val_ds and test_ds
    val_size = int(val_split / (val_split + test_split) * tf.data.experimental.cardinality(val_test_ds).numpy())
    val_ds = val_test_ds.take(val_size)
    test_ds = val_test_ds.skip(val_size)

    # Apply basic preprocessing
    normalization_layer = layers.Rescaling(1./255)
    train_ds = dataset.map(lambda x, y: (normalization_layer(x), y))
    val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))
    test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y))

    return train_ds, val_ds, test_ds

# Main execution
data_dir = r"C:/Users/91911/OneDrive/Desktop/IEEE/COFFEE/dataset"
train_ds, val_ds, test_ds = load_and_preprocess_data(data_dir)

# Calculate class weights
y_train = np.concatenate([y.numpy() for x, y in train_ds], axis=0)
class_weights = dict(enumerate(class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)))

# Implement custom cross-validation
n_folds = 5
dataset_size = tf.data.experimental.cardinality(train_ds).numpy()
fold_size = dataset_size // n_folds
fold_accuracies = []

for fold in range(n_folds):
    print(f"\nTraining fold {fold + 1}")
    
    val_start = fold * fold_size
    val_end = (fold + 1) * fold_size
    
    val_ds_fold = train_ds.skip(val_start).take(fold_size)
    train_ds_fold = train_ds.take(val_start).concatenate(train_ds.skip(val_end))
    
    # Create and train the advanced model
    advanced_model_v2 = create_advanced_model_v2()
    history, fold_acc = train_and_evaluate_model_v2(advanced_model_v2, train_ds_fold, val_ds_fold, test_ds, class_weights)
    fold_accuracies.append(fold_acc)

    # Save the model for this fold
    advanced_model_v2.save(f'coffee_leaf_model_fold_{fold+1}.h5')

# Print cross-validation results
print("\nCross-validation results:")
for i, acc in enumerate(fold_accuracies):
    print(f"Fold {i+1} accuracy: {acc:.4f}")
print(f"Mean accuracy: {np.mean(fold_accuracies):.4f} (+/- {np.std(fold_accuracies):.4f})")

# Save the best model (highest accuracy)
best_fold = np.argmax(fold_accuracies)
best_model = tf.keras.models.load_model(f'coffee_leaf_model_fold_{best_fold+1}.h5')
best_model.save('best_coffee_leaf_model_final.h5')

print(f"\nBest model (from fold {best_fold+1}) saved as 'best_coffee_leaf_model_final.h5'")

# Analyze misclassifications using the best model
test_images = np.concatenate([x.numpy() for x, _ in test_ds], axis=0)
test_labels = np.concatenate([y.numpy() for _, y in test_ds], axis=0)
predictions = best_model.predict(test_images)
misclassified = np.where(np.argmax(predictions, axis=1) != np.argmax(test_labels, axis=1))[0]

print(f"\nNumber of misclassified images: {len(misclassified)}")
print("Class distribution of misclassified images:")
class_names = ['healthy', 'red_spider_mite', 'Rust']
for class_idx, class_name in enumerate(class_names):
    count = np.sum(np.argmax(test_labels[misclassified], axis=1) == class_idx)
    print(f"{class_name}: {count}")


Found 1000 files belonging to 2 classes.
Using 700 files for training.
Found 1000 files belonging to 2 classes.
Using 300 files for validation.

Training fold 1
Epoch 1/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 146s 3s/step - accuracy: 0.4765 - loss: 6.9716 - val_accuracy: 0.6641 - val_loss: 6.6599 - learning_rate: 2.0000e-04
Epoch 2/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 72s 3s/step - accuracy: 0.5691 - loss: 6.5241 - val_accuracy: 0.6484 - val_loss: 6.1686 - learning_rate: 4.0000e-04
Epoch 3/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 60s 3s/step - accuracy: 0.5087 - loss: 5.9411 - val_accuracy: 0.5312 - val_loss: 5.3841 - learning_rate: 6.0000e-04
Epoch 4/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 55s 3s/step - accuracy: 0.5871 - loss: 5.2090 - val_accuracy: 0.6172 - val_loss: 4.6322 - learning_rate: 8.0000e-04
Epoch 5/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 54s 3s/step - accuracy: 0.5538 - loss: 4.3731 - val_accuracy: 0.5781 - val_loss: 3.7386 - learning_rate: 0.0010
Epoch 6/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 78s 3s/step - accuracy: 0.56

ValueError: Unable to synchronously create dataset (name already exists)

In [61]:
# Assuming 'model' is your trained model
model.save('best_coffee_leaf_model_final.keras')
print("Model saved as 'best_coffee_leaf_model_final.keras'")

Model saved as 'best_coffee_leaf_model_final.keras'


In [54]:
def plot_learning_curves(history):
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.tight_layout()
    plt.savefig('learning_curves.png')
    plt.close()

# After training
plot_learning_curves(history)

In [63]:
pip install flask

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


SystemExit: 1

C:\Users\91911\AppData\Roaming\Python\Python312\site-packages\IPython\core\interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [56]:
import os

print("Current working directory:", os.getcwd())
print("Files in this directory:", os.listdir())

Current working directory: c:\Users\91911\OneDrive\Desktop\IEEE\COFFEE
Files in this directory: ['bestleaf_model2.h5', 'best_coffee_leaf_model.h5', 'best_coffee_leaf_model2.h5', 'best_coffee_leaf_modelww.h5', 'best_coffee_leaf_model_1725653124.h5', 'best_coffee_leaf_model_v2.h5', 'cnn_coffee_leaf_model.h5', 'coffee.ipynb', 'coffeee.ipynb', 'coffee_leaf_model_fold_1.h5', 'dataset', 'learning_curves.png', 'ragha_coffee_leaf_model.keras', 'rag_coffee_leaf_model2.h5', 'training_history.png']
